# setup

In [1]:
import torch
import os
import numpy as np

from tqdm import tqdm

In [2]:
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

from configs.gate import LoadDataConfig, Downstream_cnn_args
# from configs.fake import LoadDataConfig, Downstream_cnn_args
from data.load_data import LoadData
from models.baseline import ResnetBaseline
from utils import synthesis

# init

In [3]:
model_label = 'gate'

In [4]:
loader_config = LoadDataConfig()
resnet_config = Downstream_cnn_args()

In [5]:
dataloader = LoadData(**loader_config.__dict__)
reference = ResnetBaseline(**resnet_config.__dict__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# threshould

In [6]:
model = torch.load('output/{}.pt'.format(model_label))
assert model.state_dict().keys() == reference.state_dict().keys()
model = model.to(device)

In [7]:
val_dl = dataloader.get_val_dataloader()
best_f1s, best_thresholds = synthesis(model, val_dl, None, device)
best_f1s, best_thresholds

100%|██████████| 272/272 [07:17<00:00,  1.61s/it]


([0.35000000000000003, 0.48, 0.0],
 [0.11551620044274503, 0.11213626685592619, 0.9424144275656523])

# test

In [8]:
test_dl = dataloader.get_test_dataloader()
all_binary_results, all_true_labels, metrics_dict = synthesis(model, test_dl, best_thresholds, device)
metrics_dict

  0%|          | 0/135 [00:00<?, ?it/s]

100%|██████████| 135/135 [03:39<00:00,  1.63s/it]


{'Accuracy': [0.06037277147487844, 0.06025700393609632, 0.11258393146561704],
 'Precision': [0.0593382395549632, 0.06025700393609632, 0.0],
 'Recall': [1.0, 1.0, 0.0],
 'F1 Score': [0.11202888244625565, 0.11366490145766228, 0.0]}